In [8]:
from google.cloud import bigquery
import os
from datetime import datetime

# Caminho para o arquivo de credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\diego\OneDrive\DATAMASTER\dados_bacen_new\case-radar-de-risco-f1ef8b6de88c.json"

# 📅 Data atual no formato YYYYMMDD
data_hoje = datetime.today().strftime('%Y%m%d')
data_formatada = datetime.today().date()  # yyyy-mm-dd

# 🔧 Parâmetros dinâmicos
bucket_name = "bucket-datamaster"
file_name = f"inadimplencia_bacen_{data_hoje}.csv"
dataset_id = "camada_bronze"
table_id = "dados_bacen"

# URI do arquivo no bucket
uri = f"gs://{bucket_name}/{file_name}"

# Cliente do BigQuery
client = bigquery.Client()

# Configuração do job
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",  # Sobrescreve se já existir
)

# Cria a tabela temporária
table_ref = client.dataset(dataset_id).table(table_id)
load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)
load_job.result()

# Adiciona a coluna data_carga com a data de execução
query = f"""
CREATE OR REPLACE TABLE `{client.project}.{dataset_id}.{table_id}` AS
SELECT *, DATE('{data_formatada}') AS data_carga
FROM `{client.project}.{dataset_id}.{table_id}`
"""

query_job = client.query(query)
query_job.result()

print(f"✅ Tabela `{table_id}` criada com sucesso em `{dataset_id}` com campo data_carga = {data_formatada}")

✅ Tabela `dados_bacen` criada com sucesso em `camada_bronze` com campo data_carga = 2025-08-15
